In [1]:
from astropy.table import Table
import numpy as np
#vt = Table.read('/Users/nicholebarry/MWA/data/2014_EoR0_all.xml', format='votable')
#vt = Table.read('/Users/nicholebarry/MWA/data/2016_EoR0_compact.xml', format='votable')
vt = Table.read('/Users/nicholebarry/MWA/zenith_data_beamdeform.xml', format='votable')

In [2]:
vt

obs_id,starttime,stoptime,duration,starttime_mjd,stoptime_mjd,starttime_utc,stoptime_utc,starttime_mro,stoptime_mro,obsname,creator,modtime,mode,projectid,projectshortname,projectdescription,dataquality,dataqualityname,dataqualitycomment,deleted_flag,deletedtime_mro,vcs_recombined_flag,int_time,freq_res,ra,dec,ra_pointing,dec_pointing,ra_phase_center,dec_phase_center,azimuth_pointing,elevation_pointing,azimuth,elevation,gal_lat,gal_long,sky_temp,calibration,calibrators,gridpoint_name,gridpoint_number,local_sidereal_time_deg,sun_elevation,sun_pointing_distance,jupiter_pointing_distance,moon_pointing_distance,channel_numbers_csv,channel_center_frequencies_mhz_csv,first_channel_number,center_channel_number,last_channel_number,first_channel_lowest_frequency_mhz,center_frequency_mhz,last_channel_highest_frequency_mhz,channels_are_contiguous,channel_count,channel_width_mhz,mwa_array_configuration,groupid,collection_id,collection_name,collection_url,iono_qa,tiles_with_bad_states,tiles_with_bad_pointings,tiles_with_bad_freqs,tiles_with_bad_gains,tiles_with_bad_beamshape,tiles_flagged,bad_tiles,good_tiles,total_tiles,vcs_files_raw_total,vcs_files_raw_archived,vcs_files_ics_archived,vcs_files_tar_archived,gpubox_files_total,gpubox_files_archived,flag_files_exist,total_archived_data_files,total_archived_data_bytes,total_archived_files,total_archived_bytes,files_deleted,files_deleted_bytes,data_files_summary,downloads,first_download_datetime_mro,last_download_datetime_mro,days_since_last_download,days_from_obs_to_first_download,downloaded_bytes,delaymode_name,delaymode_description,cable_delays,cable_delays_description,geometric_delays,geometric_delays_description,calibration_delays,oversampled_flag,deripple_flag,deripple_params
int64,int64,int64,int32,float32,float32,object,object,object,object,str255,str255,object,str50,str50,str255,str255,int32,str255,str2048,bool,object,bool,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,str255,str50,int32,float32,float32,float32,float32,float32,str128,str256,int32,int32,int32,float32,float32,float32,bool,int32,float32,object,int64,int64,object,object,float32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int64,int64,int64,int64,int64,int64,bool,int64,int64,int64,int64,int64,int64,str255,int32,object,object,int32,int32,int64,str255,str255,int32,str255,int32,str255,bool,bool,bool,str255
1252341200,1252341200,1252341320,120,58738.69,58738.69,2019-09-12T16:33:02.000Z,2019-09-12T16:35:02.000Z,2019-09-13T00:33:02.000Z,2019-09-13T00:35:02.000Z,"high_2019B_2458738_EOR0_RADec0.0,-27.0_143",andrew,2022-11-18T06:52:37.159Z,HW_LFILES,G0009,EoR,(Webster) Epoch of Reionisation,1,Good,,False,,False,2.0,40.0,356.3131,-26.813354,356.3131,-26.813354,0.0,-27.0,0.0,90.0,0.0,90.0,-78.55294,30.635677,172.59024,False,,sweet,0,356.319,-66.72147,156.66159,89.7719,23.043074,"131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154","167.68,168.96,170.24,171.52,172.8,174.08,175.36,176.64,177.92,179.2,180.48,181.76,183.04,184.32,185.6,186.88,188.16,189.44,190.72,192,193.28,194.56,195.84,197.12",131,142,154,167.04,181.12,197.76,True,24,1.28,Phase II Compact,1252332792,--,,,--,0,1,0,0,1,2,4,124,128,0,0,0,0,24,24,True,24,5072814720,26,5076615246,0,0,24,2,2022-09-21T08:55:17.940Z,2022-10-06T00:53:01.607Z,357,1104,35323740160,NODELAYS,No delays applied,0,None,0,None,False,False,False,
1252341320,1252341320,1252341440,120,58738.69,58738.69,2019-09-12T16:35:02.000Z,2019-09-12T16:37:02.000Z,2019-09-13T00:35:02.000Z,2019-09-13T00:37:02.000Z,"high_2019B_2458738_EOR0_RADec0.0,-27.0_143",andrew,2022-11-18T06:52:37.159Z,HW_LFILES,G0009,EoR,(Webster) Epoch of Reionisation,1,Good,,False,,False,2.0,40.0,356.81494,-26.813425,356.81494,-26.813425,0.0,-27.0,0.0,90.0,0.0,90.0,-78.55294,30.635677,172.59024,False,,sweet,0,356.82037,-66.60134,156.53687,90.176926,23.425459,"131,132,133,134,135,136,137,138,139,140,141,142,143,144,1

In [4]:
# Get the relevant information from the table. Turn them into numpy arrays
obs_id = vt['obs_id']
obs_id_data = np.array(obs_id.data)
gridpoint_number = vt['gridpoint_number']
gridpoint_number_data = np.array(gridpoint_number.data)
starttime_utc = vt['starttime_utc']
starttime_utc_data = np.array(starttime_utc.data)

In [5]:
#Use the difference between neighboring observations to see if there are jumps.
# These indicate new days
end_day_inds = np.argwhere(np.diff(obs_id) > 1000) 
new_day_inds = end_day_inds + 1

#Print out the hanging observations at the start and end of the table in case this
# votable table is a subset 
redo_day_start = obs_id[end_day_inds[0]] 
redo_day_end = obs_id[new_day_inds[new_day_inds.size-1]]
print("Redo start: " + str(redo_day_start.data[0])) 
print("Redo end: " + str(redo_day_end.data[0]))

#Grab the indices from the nonhanging observation chunks
end_to_print_inds = end_day_inds[1:] + 1
start_to_print_inds = new_day_inds[:new_day_inds.size-1]

#Create lists where each new entry is a new day for both the obs ids and the grid point numbers 
day_list = [] 
pointing_list = [] 
for start_i in range(len(start_to_print_inds)): 
    day = np.array(obs_id_data)[start_to_print_inds[start_i].item():end_to_print_inds[start_i].item()] 
    day_list.append(day)
    pointing = np.array(gridpoint_number_data)[start_to_print_inds[start_i].item():end_to_print_inds[start_i].item()] 
    pointing_list.append(pointing)

#Create the names of the days and pointings for the filenames.
# Optionally choose a different starting index for the day names in case this is a subset votable table
day_name_start=1  
#pointing_names = ['minustwo','minusone','zenith','plusone','plustwo'] 
pointing_names = ['zenith'] 
day_names = np.arange(len(day_list)) + day_name_start
#pointing_numbers = [3,1,0,2,4]
pointing_numbers = [0]

#Populate the by-day observation txt files, ie 1.txt, 2.txt, ...
for day_i,day_name in enumerate(day_names):
    #filename = '/Users/nicholebarry/MWA/data/obs_lists/' + str(day_name) + '.txt'
    #filename = '/Users/nicholebarry/MWA/data/obs_lists/2016/' + str(day_name) + '.txt'
    filename = '/Users/nicholebarry/MWA/data/obs_lists/zenith_deform/' + str(day_name) + '.txt'
    with open(filename, 'w') as outfile:
        for obs in day_list[day_i]:
            outfile.write(f"{obs:}\n")

    
# #Split each day into pointing subsets and create the by-pointing observation txt files for 
# # each day, ie 1_minustwo.txt, 1_minusone.txt, ...
# for day_i,day in enumerate(day_list):
#     day_name = day_names[day_i]
#     pointings_in_day = pointing_list[day_i]
#     ar, index = np.unique(pointings_in_day,return_index=True)
#     for pointing in pointings_in_day[sorted(index)]:
#         pointing_inds = np.argwhere(pointings_in_day == pointing)
#         num_poi = np.argwhere(pointing == pointing_numbers).size
#         if num_poi > 0:
#             pointing_name = pointing_names[np.argwhere(pointing == pointing_numbers).item()]
#             #filename = '/Users/nicholebarry/MWA/data/obs_lists/' + str(day_name) + '_' + str(pointing_name) + '.txt'
#             filename = '/Users/nicholebarry/MWA/data/obs_lists/zenith_deform/' + str(day_name) + '_' + str(pointing_name) + '.txt'
#             with open(filename, 'w') as outfile:
#                 for obs in day[pointing_inds]:
#                     outfile.write(f"{obs.item():}\n")
                
#For documentation purposes, also record the actual starting datetimes of the days in that dir                 
#filename = '/Users/nicholebarry/MWA/data/obs_lists/days_all.txt'
filename = '/Users/nicholebarry/MWA/data/obs_lists/zenith_deform/days_all.txt'
with open(filename, 'w') as outfile:
    for day_i,day_name in enumerate(day_names):
        day_start = starttime_utc_data[start_to_print_inds[day_i].item()]
        outfile.write(f"{day_start:}\n")


Redo start: 1252342880
Redo end: 1259408440


In [6]:
np.argwhere(np.diff(obs_id) > 1000).size

29